In [4]:
import keras#from keras.models import load_model
from tensorflow.keras import models
import numpy as np
import scipy.misc
import os,glob,cv2,shutil,argparse,matplotlib
import tensorflow as tf
from PIL import Image

In [3]:
!pip freeze

absl-py==0.14.0
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client==1.7.2
anaconda-navigator==2.0.3
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1610472525955/work
anyio @ file:///opt/concourse/worker/volumes/live/64740ac7-3a9c-4fbb-6685-a51c4ff8b4ca/volume/anyio_1617783319350/work/dist
appdirs==1.4.4
applaunchservices==0.2.1
appnope @ file:///opt/concourse/worker/volumes/live/5f13e5b3-5355-4541-5fc3-f08850c73cf9/volume/appnope_1606859448618/work
appscript @ file:///opt/concourse/worker/volumes/live/82e8b4c7-2416-4d10-509e-144ca79d9b1d/volume/appscript_1611426996703/work
argh==0.26.2
argon2-cffi @ file:///opt/concourse/worker/volumes/live/d733ceb5-7f19-407b-7da7-a386540ab855/volume/argon2-cffi_1613037492998/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///opt/concourse/worker/volumes/live/343a8902-287c-47fb-6db8-923a63364302/volume/astroid_1613500849157/work
astropy @ file:///opt/concourse/wo

# convert keras(.h5) model to tflite model

In [ ]:
import tensorflow as tf

model_path = "./models/trained_resnet_model_v2_10.h5"

model = tf.keras.models.load_model(model_path)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("./models/hw_recog_digit_tmp.tflite", "wb").write(tflite_model)

In [ ]:
TFLITE_MODEL="./models/trained_resnet_model_v2_10.tflite"

tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)
tflite_interpreter.allocate_tensors()

input_index = tflite_interpreter.get_input_details()
output_index = tflite_interpreter.get_output_details()


print("== Input details ==")
print("shape:", input_index[0]['shape'])
print("type:", input_index[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_index[0]['shape'])
print("type:", output_index[0]['dtype'])

In [ ]:
model_path = "./models/trained_resnet_model_v2_10.h5"

#model = tf.keras.models.load_model("./models/trained_resnet_model_v2_10.h5")


# benchmarking on digit level

In [ ]:
def pred_using_h5_digit(model,path):
    result = {}
    wrong_count=0
    for img1 in sorted(glob.glob(path)):
        img=cv2.imread(img1)
        img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.astype('float32') / 255.
        img=np.reshape(img,(1,28,28,1))
        res=model.predict(img)
        pred=res[0].argmax(axis=0)
        ground = int(img1.split('/')[8])
        name = img1.split('/')[-2]
        if pred!=ground:
            result[name]=pred
            wrong_count+=1
    accuracy = (len(glob.glob(path))-wrong_count)/len(glob.glob(path))
    return result,accuracy

In [ ]:
path = "./testing_data/test/8/*.jpg"
result,accuracy = pred_using_h5_digit(model,path)

# benchmarking for roll number


In [ ]:
def pred_using_h5(model,path):
    result = {}
    
    for roll in glob.glob(path):
        ground_roll = str(roll.split('/')[8])
        pred_roll = ""
        int_med_res = {}
        for img1 in sorted(glob.glob(path.split('*')[0]+ground_roll+'/*.jpg')):
            img=cv2.imread(img1)
            img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = img.astype('float32') / 255.
            img=np.reshape(img,(1,28,28,1))
            res=model.predict(img)
            pred=res[0].argmax(axis=0)
            pred_roll=pred_roll+str(pred)
            name = img1.split('/')[9]
            int_med_res[name] = pred
        if ground_roll==pred_roll:
            result[ground_roll]=1
        else:
            print(ground_roll,  int_med_res)
            result[ground_roll]=0

    return result

In [ ]:
path = "./testing_data/site-data/*"
#result = pred_using_h5(model,path)

In [ ]:
from collections import Counter
res = Counter(result.values())
res


# test model using tflite model

In [ ]:
test_img_dir = "./gujrat_sever/byte_2_images/*/"

def test_tflite(img_path):
    result = {}
    wrong_count=0; correct_count=0
    for roll in sorted(glob.glob(img_path)):
        for img1 in sorted(glob.glob(roll+"/*.jpg")):
            img_name= img1.split("/")[-1]
            img=cv2.imread(img1)
            img=cv2.resize(img,(28,28))
            img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = np.array(img,dtype=np.float32) / 255.0
            img=np.reshape(img,(1,28,28,1))
            tflite_interpreter.set_tensor(input_index[0]['index'],img)
            tflite_interpreter.invoke()
            predictions = tflite_interpreter.get_tensor(output_index[0]['index'])
            pred=predictions[0].argmax(axis=0)
            label = img_name.split("__")[0]   ############## to get label of the image
            if int(label)!=pred:
                wrong_count+=1
                result[img_name]={"ground":label,"prediction":pred}
            else:
                correct_count+=1 
    
    accuracy = (len(glob.glob(img_path+"/*.jpg"))-wrong_count)*100/len(glob.glob(img_path+"/*.jpg"))
    return result,accuracy,correct_count
    

In [ ]:
result,accuracy,correct_count = test_tflite(test_img_dir)

In [ ]:
result

In [ ]:
correct_count

# Data Augmentation for visualization purpose

In [ ]:
#augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
image_dir = './test_data2/*.jpg'
save_dir  = './credentials/test_data_augmented/'
datagen = ImageDataGenerator(
        rotation_range=30,
        zoom_range=5,
        width_shift_range=0.5,
        height_shift_range=0.5,
        shear_range=0.5,
        fill_mode='nearest')
for i in sorted(glob.glob(image_dir)):
    img=load_img(i)
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # 
    j = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=save_dir, save_format='jpg'):
        j += 1
        if j > 3:
            break  # otherwise the generator would loop indefinitely

# Read json file to get base64 image 

In [ ]:
import json
json_file_path="./data/gujrat_prod_latest.json"
data= open(json_file_path,"r")

In [ ]:
data2=json.loads(data.read())


In [ ]:
##### save image
import uuid
base_path="./gujrat_sever/byte_2_images/"
def convert_base_2_image(json_data):
    for s_id in json_data:
        save_path=base_path+str(s_id['studentId'])
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        
        for idx,d in enumerate(s_id['studentIdTrainingData']):
            gt_name = str(s_id['studentId'])[idx]
            imgdata = base64.b64decode(d)
            filename = save_path+"/"+str(gt_name)+"__"+str(uuid.uuid4())+'.jpg'  # I assume you have a way of picking unique filenames
            with open(filename, 'wb') as f:
                f.write(imgdata)
        
            

In [ ]:
convert_base_2_image(data2)